In [117]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import copy
import random

# Camiones y Camión

In [17]:
df_clientes = pd.read_excel("data/inputs/data_inputs.xlsx", sheet_name="clientes")
df_pedidos = pd.read_excel("data/inputs/data_inputs.xlsx", sheet_name="pedidos")
df_pedidos = pd.merge(left=df_pedidos, right=df_clientes, on="cliente", how="inner")
df_pedidos

,cliente,pedidos1,pedidos2,pedidos3,pedidos4,pedidos5,pedidos6,coord_x,coord_y
0,A,4,3,5,2,6,6,0.257571,1.803726
1,B,6,6,5,4,1,3,1.523313,2.102301
2,C,1,6,6,2,6,7,0.710910,2.629800
3,D,8,6,5,2,8,2,1.013940,2.149740
4,E,4,1,2,4,2,3,0.609480,0.666810
5,F,3,1,6,8,6,5,0.998640,1.451790
6,G,3,2,3,7,4,3,1.434510,0.018180
7,H,5,8,5,2,4,8,2.171880,0.764910
8,I,5,2,3,4,2,6,1.412280,0.863190
9,J,1,8,6,8,1,2,2.386260,1.387260


In [18]:
df_camiones = pd.read_excel("data/inputs/data_inputs.xlsx", sheet_name="camiones")
df_camiones

,camion,carga_max,pedidos_max,dist_max
0,1,12,3,2
1,2,12,3,2
2,3,12,3,2
3,4,12,3,2
4,5,12,3,2
5,6,12,3,2


In [165]:
class Camion(object):
    """ 
    La clase Camión permite generar instancias de camiones con sus respectivas características:
        - ix: Identificador del camión.
        - carga_max: Máxima carga admitida.
        - pedidos_max: Máximo número de pedidos admitidos.
        - dist_max: Máxima distancia entre clientes.
        
    Dentro de pedidos_asignados se guardan todos los pedidos que se agregan al camión acompañados del ix del pedido.
    Se puede acceder fácilmente a la carga total actual del camion y cantidad de pedidos con esos atributos.
    """
    
    def __init__(self, ix, carga_max, pedidos_max, dist_max):
        self.ix = ix
        self.carga_max = carga_max
        self.pedidos_max = pedidos_max
        self.dist_max = dist_max
        self.pedidos_asignados = {}
        self.carga_total = 0
        self.cantidad_pedidos = 0
        
        
    def __str__(self):
        return f'Camión {self.ix}\nCarga Total {self.carga_total} tn\nPedidos {[ped.ix for ped in self.pedidos_asignados.values()]}\nCosto Total {self.get_costo()}\nCosto por tn {self.get_costo_tn()}'
    
    def __repr__(self):
        return f'Camión {self.ix}\nCarga Total {self.carga_total} tn\nPedidos {[ped.ix for ped in self.pedidos_asignados.values()]}\nCosto Total {self.get_costo()}\nCosto por tn {self.get_costo_tn()}'
    
    def _check_pedido_carga(self, pedido):
        """ 
        Permite pasarle al camión un nuevo pedido y revisar carga máxima.
            - Devuelve True si sumando el nuevo pedido no excedemos la carga máxima.
            - Devuelve False si sumando el nuevo pedido excedemos la carga máxima.
        """
        return (self.carga_total + pedido.carga) <= self.carga_max
    
    def _check_pedido_cantidad(self):
        """ 
        Permite revisar si se alcanzó la cantidad de pedidos máxima.
            - Devuelve True si sumando el nuevo pedido no excedemos la cantidad de pedidos máxima.
            - Devuelve False si sumando el nuevo pedido excedemos la cantidad de pedidos máxima.
        """
        return self.cantidad_pedidos < self.pedidos_max
    
    def _check_pedido_distancia(self, pedido):
        """ 
        Permite pasarle al camión un nuevo pedido y revisar que su distancia no supere la distancia máxima
        con el resto de pedidos.
            - Devuelve True si sumando el nuevo pedido no excedemos la distancia máxima para ningún pedido.
            - Devuelve False si sumando el nuevo pedido excedemos la distancia máxima en al menos un pedido.
        """
        dist_check = [pedido.distancia(other) <= self.dist_max for other in self.pedidos_asignados.values()]
        return all(dist_check)
    
    def check_nuevo_pedido(self, pedido):
        """ 
        Permite pasarle al camión un nuevo pedido y revisar si puede agregarse al camión cumpliendo
        la carga máxima, distancia máxima y cantidad de pedidos máxima.
            - Devuelve True si sumando el nuevo pedido no excedemos ninguna restricción.
            - Devuelve False si sumando el nuevo pedido excedemos alguna restricción.
        """
        #return not pedido.asignado and self._check_pedido_carga(pedido) and self._check_pedido_cantidad(pedido) and self._check_pedido_distancia(pedido)
        return self._check_pedido_carga(pedido) and self._check_pedido_cantidad() and self._check_pedido_distancia(pedido)
        
    def check_intercambio_pedido(self, pedido):
        """ 
        Permite pasarle al camión un pedido que no pertenezca al mismo y revisar a 
        que pedidos ya asignados podría reemplazar. Esta comparación se realiza con el pedido que se pasa al método
        contra todos los pedidos asignados en el camión.
            
        Que un nuevo pedido pueda reemplazar a otro implica que:
            - Eliminando la carga del pedido original (a reemplazar) y agregando la nueva carga no se supera la carga máxima.
            - La distancia del nuevo pedido con los pedidos restantes del camión (sin tener en cuenta el pedido a reemplazar) no supera la distancia máxima.
            
        Returns:
            list: Lista de ix de pedidos que podrían ser reemplazados por el pedido pasado como argumento del método.
        """    
        
        # Revisamos que el pedido no esté asignado previamente al camión.
        if pedido.ix not in self.get_ix_pedidos():
            
            # Creamos una lista de ixs reemplazables por el pedido nuevo.
            ix_pedidos_reemplazables = []
            
            # Para cada uno de los pedidos ya asignado al camión se hace el chequeo.
            for pedido_original in self.get_pedidos():
                # Calculamos la carga nueva que tendría el camión con el reemplazo de pedidos.
                nueva_carga = self.carga_total + pedido.carga - pedido_original.carga
                # Generamos una lista de bool revisando si el nuevo pedido incorporado supera la distancia máxima con los pedidos restantes en el camión.
                distancias = [pedido.distancia(pedido_restante) <= self.dist_max for pedido_restante in self.get_pedidos() if pedido_restante.ix != pedido_original.ix]
                
                # Si la nueva carga no supera la carga máxima y las distancias con pedidos restantes son todas menores al máximo
                # agregamos el ix del pedido a reemplazar en la lista.
                if nueva_carga <= self.carga_max and all(distancias):
                    ix_pedidos_reemplazables.append(pedido_original.ix)
                    
            return ix_pedidos_reemplazables
        
        # Si el pedido ya estaba asignado al camión se devuelve una lista vacía.
        else:
            return []
        
        
    def add_pedido(self, pedido):
        """
        Permite pasarle al camión un pedido no asignado previamente e incorporarlo al mismo.
        Este método no realiza los chequeos de las restricciones del camión.
        """
        if not pedido.asignado:
            # Se agrega el pedido al diccionario de pedidos asignados de este camión.
            self.pedidos_asignados[pedido.ix] = pedido
            # Se actualiza la carga total y cantidad de pedidos totales.
            self.carga_total += pedido.carga
            self.cantidad_pedidos += 1
            # Se actualizan las propiedades del pedido agregado.
            pedido.asignado = True
            pedido.camion_ix = self.ix
        #     return True
        # else:
        #     return False
            
    def add_pedido_checked(self, pedido):
        """
        Permite pasarle al camión un pedido no asignado previamente e incorporarlo al mismo.
        A diferencia del método add_pedido() este método sólo agrega el pedido si pasa los chequeos de las restricciones.
        """
        if self.check_nuevo_pedido(pedido):
            self.add_pedido(pedido)
        
    
    def remove_pedido(self, pedido_ix):
        """
        Permite pasarle al camión un ix de un pedido asignado y eliminarlo del diccionario
        de pedidos asignados.
        """
        # Confirma que el pedido esté asignado a este camión.
        if pedido_ix in self.get_ix_pedidos():
            # Actualiza los parámetros del pedido.
            self.get_pedido(pedido_ix).asignado = False
            self.get_pedido(pedido_ix).camion_ix = None
            # Actualiza los parámetros del camión.
            self.carga_total -= self.pedidos_asignados.get(pedido_ix).carga
            self.cantidad_pedidos -= 1
            # Elimina el pedido.
            self.pedidos_asignados.pop(pedido_ix)
        else:
            print(f"El pedido {pedido_ix} no está en el camión {self.ix}")    

    def get_ix_pedidos(self):
        """
        Returns:
            list: Lista de ix de pedidos asignados al camión.
        """        
        return list(self.pedidos_asignados.keys())
    
    def get_pedidos(self):
        """
        Returns:
            list: Lista de pedidos asignados al camión.
        """     
        return list(self.pedidos_asignados.values())
    
    def get_pedido(self, ix):
        """
        Args:
            ix (int or str): Código identificador del pedido que queremos obtener de los pedidos asignados al camión.

        Returns:
            Pedido: Pedido ix asignado al camión.
        """
        return self.pedidos_asignados.get(ix)
    
    def get_ix(self):
        """
        Returns:
            int or str: Identificador ix del camión.
        """
        return self.ix
    
    def get_carga_total(self):
        """
        Returns:
            int: Carga total de pedidos asignados del camión.
        """       
        return self.carga_total
    
    def count_pedidos(self):
        """
        Returns:
            int: Cantidad de pedidos asignados del camión.
        """          
        return self.cantidad_pedidos
    
    def get_costo(self):
        """
        Returns:
            int: Costo del camión en función de la carga total.
        """   
         
        if self.carga_total == 0:
            costo = 5000
        elif self.carga_total <= 4:
            costo = 5600
        elif self.carga_total > 4 and self.carga_total < 6.5:
            costo = 1400*self.carga_total
        elif self.carga_total >= 6.5 and self.carga_total < 9.5:
            costo = 1200*self.carga_total
        else:
            costo = 1000*self.carga_total
                
        return costo 
    
    def get_costo_tn(self):
        """
        Returns:
            float: Costo por tn del camión en función de la carga total.
        """   
        if self.carga_total != 0:
            return self.get_costo()/self.carga_total
        else:
            return None

    def reset_pedidos(self):
        """
        Elimina todos los pedidos asignados del camión.
        """
        self.pedidos_asignados.clear()
        self.carga_total = 0
        self.cantidad_pedidos = 0
        
        
        
        
        
class Pedido(object):
    """ 
    La clase Pedido permite generar instancias de pedidos de clientes con sus respectivas características:
        - ix: Identificador del pedido.
        - x: Coordenada x de localización.
        - y: Coordenada y de localización.
        - carga: Carga del pedido.
        
    Si el pedido está asignado dicho parámetro toma valor True. El parámetro camion_ix indica el ix del camión al 
    que el pedido está asignado.
    """
    
    def __init__(self, ix, x, y, carga):
        self.ix = ix
        self.x = x
        self.y = y
        self.carga = carga
        self.asignado = False
        self.camion_ix = None
        
    def __str__(self):
        return f'Pedido {self.ix}\nCarga {self.carga} tn\nAsignado {self.asignado}\nAsignado a Camion {self.camion_ix}'
    
    def __repr__(self):
        return f'Pedido {self.ix}\nCarga {self.carga} tn\nAsignado {self.asignado}\nAsignado a Camion {self.camion_ix}'
    
    def get_ix(self):
        """
        Returns:
            int or str: Identificador ix del pedido.
        """
        return self.ix
    
    def get_carga(self):
        """
        Returns:
            float: Carga del pedido.
        """
        return self.carga
    
    def distancia(self, other):
        """Calcula la distancia entre la instancia del pedido y un segundo pedido pasado como argumento other.

        Args:
            other (Pedido): Pedido con el que queremos obtener la distancia respecto de la instancia actual.

        Returns:
            float: Distancia entre la instancia del pedido y el pedido en el argumento del método.
        """
        dist = ((self.x - other.x)**2 + (self.y - other.y)**2)**(1/2)
        return round(dist, 1)
    
    
    
class Ruteo(object):
    """ 
    La clase Ruteo contiene toda la información de camiones disponibles y pedidos requeridos:
        - camiones: Diccionario de camiones disponibles identificados por su ix.
        - pedidos: Diccionario de pedidos disponibles identificados por su ix.
        - costo de oportunidad: Costo de pedidos no asignados en $/tn.
        - presupuesto: Presupuesto previsto en $/tn.
        - random_state: Permite definir la semilla para la generación de valores aleatorios.
    """
    
    def __init__(self, df_camiones, df_pedidos, costo_oportunidad, presupuesto, random_state):
        self.camiones = self._load_camiones(df_camiones)
        self.pedidos = self._load_pedidos(df_pedidos)
        self.costo_oportunidad = costo_oportunidad
        self.presupuesto = presupuesto
        
        # Uso el random state para determinar la generación de solución inicial.
        self.random_state = random_state
        random.seed(self.random_state)
        
    def _load_camiones(self, df_camiones):
        """
        Este método genera el diccionario de camiones a partir de un DataFrame.

        Args:
            df_camiones (pd.DataFrame): Dataframe que contiene la información de camiones.
                                        El mismo debe contener las siguientes columnas:
                                        - camion: Contiene el identificador ix del camión.
                                        - carga_max: Carga máxima admitida para cada camión.
                                        - pedidos_max: Cantidad de pedidos máximo admitido para cada camión.
                                        - dist_max: Distancia máxima entre pedidos asignados a un camión.

        Returns:
            dict: Diccionario con los camiones del ruteo.
        """
        dict_camiones = {row.camion:Camion(ix=row.camion, carga_max=row.carga_max, pedidos_max=row.pedidos_max, dist_max=row.dist_max) for _, row in df_camiones.iterrows()}
        return dict_camiones

    def _load_pedidos(self, df_pedidos):
        """
        Este método genera el diccionario de pedidos a partir de un DataFrame.

        Args:
            df_pedidos (pd.DataFrame): Dataframe que contiene la información de pedidos.
                                        El mismo debe contener las siguientes columnas:
                                        - cliente: Contiene el identificador ix del pedido.
                                        - pedidos: Carga de cada pedido.
                                        - coord_x: Coordenada x del cliente.
                                        - coord_y: Coordenada y del cliente.

        Returns:
            dict: Diccionario con los pedidos del ruteo.
        """
        dict_pedidos = {row.cliente:Pedido(ix=row.cliente, x=row.coord_x, y=row.coord_y, carga=row.pedidos) for _, row in df_pedidos.iterrows() if row.pedidos != 0}
        return dict_pedidos
    
    def get_ix_camiones(self):
        """
        Returns:
            list: Lista de ix de camiones.
        """   
        return list(self.camiones.keys())
    
    def get_ix_pedidos(self):
        """
        Returns:
            list: Lista de ix de pedidos.
        """  
        return list(self.pedidos.keys())
    
    def get_camiones(self):
        """
        Returns:
            list: Lista de camiones.
        """  
        return list(self.camiones.values())
    
    def get_pedidos(self):
        """
        Returns:
            list: Lista de pedidos.
        """          
        return list(self.pedidos.values())
    
    def get_camion(self, ix):
        """
        Returns:
            Camion: Camion con identificador ix.
        """  
        return self.camiones.get(ix)
    
    def get_pedido(self, ix):
        """
        Returns:
            Pedido: Pedido con identificador ix.
        """  
        return self.pedidos.get(ix)
    
    def count_camiones(self):
        """
        Returns:
            int: Cantidad de camiones.
        """  
        return len(self.camiones)
    
    def count_pedidos(self):
        """
        Returns:
            int: Cantidad de pedidos.
        """  
        return len(self.pedidos)
    
        
    def get_solucion_inicial(self, mode):
        """
        Permite generar una solución inicial, es decir, realizar una asignación inicial de pedidos en camiones.
        
        Permite usar varios modos diferentes de generación de solución inicial con el argumento de este método.

        Args:
            mode (_type_): _description_
        """
        
        if mode == "simple":
            self._get_solucion_inicial_simple()
                    
        elif mode == "random":
            self._get_solucion_inicial_random()     
            
        else:
            self._get_solucion_inicial_simple()
    
    # EXPLICAR Y CHEQUEAR BIEN LAS SOLUCIONES INICIALES.              
    
    def _get_solucion_inicial_simple(self):
        """
        Genera una solución inicial deterministica. 
        Para cada camión, toma cada pedido disponible y lo intenta asignar, siendo asignado si pasa los chequeos.
        Queda definido por el orden de carga de camiones y pedidos.
        """
        for camion in self.get_camiones():
            for pedido in self.get_pedidos():
                camion.add_pedido_checked(pedido)
                
                
    def _get_solucion_inicial_random(self):
        """
        Genera una solución con aleatoriedad.
            1. Mezcla los ix de pedidos de manera aleatoria.
            2. Para cada pedido en el orden aleatorio genera una lista de ix de camiones aleatoria
            3. Para cada camión en orden aleatorio se intenta agregar el pedido. En caso de lograrse
               pasa al siguiente pedido en orden aleatorio volviendo a 2.
            
        Prueba introducir todos los pedidos en todos los camiones en orden aleatorio.
        """
        # Identificadores aleatorios de pedidos.
        ix_pedidos_rnd = random.sample(self.get_ix_pedidos(), self.count_pedidos())
        
        for ix_pedido in ix_pedidos_rnd:
            pedido = self.get_pedido(ix_pedido)
            ix_camiones_rnd = random.sample(self.get_ix_camiones(), self.count_camiones())
            
            for ix_camion in ix_camiones_rnd:
                self.get_camion(ix_camion).add_pedido_checked(pedido)
                
                if pedido.asignado:
                    break
    
    
    def generar_vecino(self, prob=1):
        """
        Realiza una modificación en la instancia de la solución, creando una nueva solución similar y válida de ruteo.
        
            1. Se selecciona un pedido al azar entre todos los pedidos. Este es el pedido a modificar (pedido_mod).
            2. Chequeamos si el pedido_mod puede ingresar a un camión de manera directa.
            
            3. Si el pedido ya está asignado:
                a. Obtenemos el ix del camión al que estaba asignado (camion al que le iría el nuevo pedido en caso de reemplazo).
                b. Eliminamos el pedido_mod del camión.
                
                4. Si el pedido_mod entra directamente en otro camion y prob es mayor al valor aleatorio uniforme(0,1):
                   Con prob = 1 (default) el 100% de las veces que entre directamente irá a esos camiones.
                    a. Se lo asigna directamente a alguno de los camiones directos al azar.
                       Si entra de manera directa no hace falta chequear que entre. 
                       No hace falta intercambiar por otro pedido porque entra directo.
                      
                4. Si el pedido no entra directamente en otro camion:
                    a. Se genera una lista de ix de pedidos que podrían ser reemplazados en sus camiones por pedido_mod.
                    b. Se genera una lista de ix de pedidos que podrían ser reemplazados y además que pueden entrar en 
                       el camión en el que estaba pedido_mod.
                    
                    5. Si hay al menos 1 pedido reemplazable posible:
                        a. Se toma un pedido al azar de estos pedidos reemplazables posibles.
                        b. Se elimina al pedido_reemplazo de su camion.
                        c. Se asigna el pedido_mod al camión del pedido_reemplazo.
                        d. Se asigna el pedido_reemplazo en el camión de pedido_mod.
                        
                    5. Si no hay al menos 1 pedido reemplazable posible:
                        a. Se devuelve pedido_mod a su camión original.
                        
            3. Si el pedido no está asignado:
            
                4. Si el pedido_mod entra directamente en otro camion:
                    a. Se lo asigna directamente a alguno de los camiones directos al azar.
                       Si entra de manera directa no hace falta chequear que entre.
                       No hace falta intercambiar por otro pedido porque entra directo.
                       
                4. Si el pedido no entra directamente en otro camion:
                    a. Se genera una lista de ix de pedidos que podrían ser reemplazados en sus camiones por pedido_mod.

                    5. Si hay al menos 1 pedido reemplazable:
                        a. Se toma un pedido al azar de estos pedidos reemplazables.
                        b. Se elimina al pedido_reemplazo de su camion.
                        c. Se asigna el pedido_mod al camión del pedido_reemplazo. El pedido_reemplazo queda sin asignar.

        """
        
        ix_pedido_mod = random.choice(self.get_ix_pedidos())
        pedido_mod = self.get_pedido(ix_pedido_mod)
        
        # Índices de los camiones en los que podría entrar directamente.
        ix_camion_directo = [camion.ix for camion in self.get_camiones() if camion.check_nuevo_pedido(pedido_mod)]
        
        # Si el pedido_mod ya está seleccionado:
        if pedido_mod.asignado:
            
            # Obtenemos el ix del camion al que pertenece pedido_mod
            ix_camion_mod = pedido_mod.camion_ix
            # Eliminamos de su camión a pedido_mod.
            self.get_camion(ix_camion_mod).remove_pedido(pedido_mod.ix)
            
            # Si el pedido_mod entra de manera directa en otro camión, es asignado de manera directa a alguno de esos camiones al azar.
            if len(ix_camion_directo) > 0 and prob >= random.uniform(0,1):
                
                # Se elige un camión directo para cambiar 
                ix_camion_new = random.choice(ix_camion_directo)
                # No hace falta chequear de que entre porque fue revisado previamente.
                self.get_camion(ix_camion_new).add_pedido(pedido_mod)
                
            # Si el pedido no entra directamente en un camión debemos reemplazar pedidos.
            else:
                
                ix_pedidos_reemplazables = []
                
                # Obtengo todos los ix de pedidos que podrían ser reemplazados por pedido_mod en sus camiones.
                for camion in self.get_camiones():
                    ix_pedidos_reemplazables += camion.check_intercambio_pedido(pedido_mod)
                               
                ix_pedidos_reemplazables_posibles = []
                
                # Para que sea posible el intercambio tenemos que quedarnos con los ix de pedidos
                # que entrarían en el camión donde estaba pedido_mod.                
                for ix_pedido_reemplazable in ix_pedidos_reemplazables:
                    pedido_reemplazable = self.get_pedido(ix_pedido_reemplazable)
                    if self.get_camion(ix_camion_mod).check_nuevo_pedido(pedido_reemplazable):
                        ix_pedidos_reemplazables_posibles += ix_pedido_reemplazable
                        
                # Si tengo al menos un reemplazo posible.
                if len(ix_pedidos_reemplazables_posibles) > 0:
                    pedido_reemplazo = self.get_pedido(random.choice(ix_pedidos_reemplazables_posibles))
                    ix_camion_new = pedido_reemplazo.camion_ix
                    self.get_camion(ix_camion_new).remove_pedido(pedido_reemplazo.ix)
                    
                    self.get_camion(ix_camion_new).add_pedido(pedido_mod)
                    self.get_camion(ix_camion_mod).add_pedido(pedido_reemplazo)
                    
                # Si no tengo ningún reemplazo posible reasigno pedido_mod a su camión original.
                else:
                    self.get_camion(ix_camion_mod).add_pedido_checked(pedido_mod)

        # Si el pedido no está asignado:
        else:
            
            # Si el pedido_mod entra de manera directa en otro camión, es asignado de manera directa a alguno de esos camiones al azar.
            if len(ix_camion_directo) > 0:
                
                # Se elige un camión directo para cambiar 
                ix_camion_new = random.choice(ix_camion_directo)
                # No hace falta chequear de que entre porque fue revisado previamente.
                self.get_camion(ix_camion_new).add_pedido(pedido_mod)
                
            # Si el pedido no entra directamente en un camión debemos reemplazar pedidos.   
            else:
                
                ix_pedidos_reemplazables = []
                
                # Obtengo todos los ix de pedidos que podrían ser reemplazados por pedido_mod en sus camiones.
                for camion in self.get_camiones():
                    ix_pedidos_reemplazables += camion.check_intercambio_pedido(pedido_mod)

                # QUE PASA SI NO TENGO NINGUNO
                if len(ix_pedidos_reemplazables) > 0:
                    pedido_reemplazo = self.get_pedido(random.choice(ix_pedidos_reemplazables))
                    ix_camion_new = pedido_reemplazo.camion_ix
                    self.get_camion(ix_camion_new).remove_pedido(pedido_reemplazo.ix)
                    
                    self.get_camion(ix_camion_new).add_pedido(pedido_mod)
             
    
    # PARA CAMBIAR TODA ESTA PARTE
    # GENERAR UN DATAFRAME CON TODOS LOS DATOS.
    def _set_results(self):
        self._set_carga_total()
        self._set_costo_camiones()
        self._set_costo_no_asignados()
        self._set_costo_total()
        self._set_costo_total_tn()
        self._set_ahorro()
        
    def get_results(self):
        self._set_results()
        print("--Resultados--")
        print(f"Carga Total {self.carga_total} tn")
        print(f"Costo Camiones {self.costo_camiones} $")
        print(f"Costo Oportunidad {self.costo_no_asignados} $")
        print(f"Costo Total {self.costo_total} $")
        print(f"Costo Total por tn {self.costo_total_tn} $")
        print(f"Ahorro {self.ahorro*100}%")
    
    def _set_carga_total(self):
        self.carga_total = sum([camion.get_carga_total() for camion in self.camiones.values()])
        
    def _set_costo_camiones(self):
        self.costo_camiones = sum([camion.get_costo() for camion in self.camiones.values()])
        
    def _set_costo_no_asignados(self):
        carga_no_asignada = sum([pedido.get_carga() for pedido in self.pedidos.values() if not pedido.asignado])
        self.costo_no_asignados = carga_no_asignada*self.costo_oportunidad
        
    def _set_costo_total(self):
        self.costo_total = self.costo_camiones + self.costo_no_asignados
    
    def _set_costo_total_tn(self):
        self.costo_total_tn = round(self.costo_total/self.carga_total, 2)
        
    def _set_ahorro(self):
        self.ahorro = round((self.costo_total_tn - self.presupuesto)/self.presupuesto, 4)

In [166]:
df_pedidos_input = (df_pedidos
                    [["cliente", "pedidos3", "coord_x", "coord_y"]]
                    .rename({"pedidos3":"pedidos"}, axis=1))

In [167]:
ruteo = Ruteo(df_camiones, df_pedidos_input, 3000, 1200, 41)
ruteo.get_solucion_inicial(mode="simple")

In [168]:
ruteo.get_results()

--Resultados--
Carga Total 55 tn
Costo Camiones 61800 $
Costo Oportunidad 0 $
Costo Total 61800 $
Costo Total por tn 1123.64 $
Ahorro -6.36%


In [169]:
ruteo.get_camiones()

[Camión 1
 Carga Total 12 tn
 Pedidos ['A', 'B', 'E']
 Costo Total 12000
 Costo por tn 1000.0,
 Camión 2
 Carga Total 11 tn
 Pedidos ['C', 'D']
 Costo Total 11000
 Costo por tn 1000.0,
 Camión 3
 Carga Total 12 tn
 Pedidos ['F', 'G', 'I']
 Costo Total 12000
 Costo por tn 1000.0,
 Camión 4
 Carga Total 11 tn
 Pedidos ['H', 'J']
 Costo Total 11000
 Costo por tn 1000.0,
 Camión 5
 Carga Total 9 tn
 Pedidos ['K', 'L']
 Costo Total 10800
 Costo por tn 1200.0,
 Camión 6
 Carga Total 0 tn
 Pedidos []
 Costo Total 5000
 Costo por tn None]

In [159]:
ruteo.get_pedidos()

[Pedido A
 Carga 5 tn
 Asignado True
 Asignado a Camion 1,
 Pedido B
 Carga 5 tn
 Asignado True
 Asignado a Camion 1,
 Pedido C
 Carga 6 tn
 Asignado True
 Asignado a Camion 2,
 Pedido D
 Carga 5 tn
 Asignado True
 Asignado a Camion 2,
 Pedido E
 Carga 2 tn
 Asignado True
 Asignado a Camion 1,
 Pedido F
 Carga 6 tn
 Asignado True
 Asignado a Camion 3,
 Pedido G
 Carga 3 tn
 Asignado True
 Asignado a Camion 3,
 Pedido H
 Carga 5 tn
 Asignado True
 Asignado a Camion 4,
 Pedido I
 Carga 3 tn
 Asignado True
 Asignado a Camion 3,
 Pedido J
 Carga 6 tn
 Asignado True
 Asignado a Camion 4,
 Pedido K
 Carga 6 tn
 Asignado True
 Asignado a Camion 5,
 Pedido L
 Carga 3 tn
 Asignado True
 Asignado a Camion 5]

In [170]:
for i in tqdm(range(5000)):
    ruteo.generar_vecino()

100%|██████████| 5000/5000 [00:00<00:00, 5852.55it/s]


In [171]:
ruteo.get_pedidos()

[Pedido A
 Carga 5 tn
 Asignado True
 Asignado a Camion 1,
 Pedido B
 Carga 5 tn
 Asignado True
 Asignado a Camion 4,
 Pedido C
 Carga 6 tn
 Asignado True
 Asignado a Camion 1,
 Pedido D
 Carga 5 tn
 Asignado True
 Asignado a Camion 2,
 Pedido E
 Carga 2 tn
 Asignado True
 Asignado a Camion 3,
 Pedido F
 Carga 6 tn
 Asignado True
 Asignado a Camion 4,
 Pedido G
 Carga 3 tn
 Asignado True
 Asignado a Camion 3,
 Pedido H
 Carga 5 tn
 Asignado True
 Asignado a Camion 3,
 Pedido I
 Carga 3 tn
 Asignado True
 Asignado a Camion 5,
 Pedido J
 Carga 6 tn
 Asignado True
 Asignado a Camion 6,
 Pedido K
 Carga 6 tn
 Asignado True
 Asignado a Camion 6,
 Pedido L
 Carga 3 tn
 Asignado True
 Asignado a Camion 2]

In [172]:
ruteo.get_results()

--Resultados--
Carga Total 55 tn
Costo Camiones 59200 $
Costo Oportunidad 0 $
Costo Total 59200 $
Costo Total por tn 1076.36 $
Ahorro -10.299999999999999%


In [163]:
ruteo.get_camiones()

[Camión 1
 Carga Total 12 tn
 Pedidos ['E', 'H', 'D']
 Costo Total 12000
 Costo por tn 1000.0,
 Camión 2
 Carga Total 11 tn
 Pedidos ['J', 'B']
 Costo Total 11000
 Costo por tn 1000.0,
 Camión 3
 Carga Total 12 tn
 Pedidos ['L', 'I', 'F']
 Costo Total 12000
 Costo por tn 1000.0,
 Camión 4
 Carga Total 9 tn
 Pedidos ['G', 'K']
 Costo Total 10800
 Costo por tn 1200.0,
 Camión 5
 Carga Total 11 tn
 Pedidos ['A', 'C']
 Costo Total 11000
 Costo por tn 1000.0,
 Camión 6
 Carga Total 0 tn
 Pedidos []
 Costo Total 5000
 Costo por tn None]

In [155]:
ruteo2 = copy.deepcopy(ruteo)

In [121]:
for camion in ruteo.camiones.values():
    print(camion)
    print("------------")
    
for pedido in ruteo.pedidos.values():
    print(pedido)
    print("------------")
    
print("-------------")
ruteo.get_results()

Camión 1
Carga Total 6 tn
Pedidos ['J']
Costo Total 8400
Costo por tn 1400.0
------------
Camión 2
Carga Total 10 tn
Pedidos ['H', 'B']
Costo Total 10000
Costo por tn 1000.0
------------
Camión 3
Carga Total 9 tn
Pedidos ['I', 'K']
Costo Total 10800
Costo por tn 1200.0
------------
Camión 4
Carga Total 11 tn
Pedidos ['C', 'A']
Costo Total 11000
Costo por tn 1000.0
------------
Camión 5
Carga Total 8 tn
Pedidos ['D', 'L']
Costo Total 9600
Costo por tn 1200.0
------------
Camión 6
Carga Total 11 tn
Pedidos ['F', 'E', 'G']
Costo Total 11000
Costo por tn 1000.0
------------
Pedido A
Carga 5 tn
Asignado True
Asignado a Camion 4
------------
Pedido B
Carga 5 tn
Asignado True
Asignado a Camion 2
------------
Pedido C
Carga 6 tn
Asignado True
Asignado a Camion 4
------------
Pedido D
Carga 5 tn
Asignado True
Asignado a Camion 5
------------
Pedido E
Carga 2 tn
Asignado True
Asignado a Camion 6
------------
Pedido F
Carga 6 tn
Asignado True
Asignado a Camion 6
------------
Pedido G
Carga 3 tn
A

In [123]:
ruteo2 = copy.deepcopy(ruteo)
print(ruteo2.get_camion(1))
print(ruteo2.get_pedido("J"))
print("------")
ruteo2.get_camion(1).remove_pedido("J")
print(ruteo2.get_camion(1))
print(ruteo2.get_pedido("J"))
print("------")
print(ruteo.get_camion(1))
print(ruteo.get_pedido("J"))

Camión 1
Carga Total 6 tn
Pedidos ['J']
Costo Total 8400
Costo por tn 1400.0
Pedido J
Carga 6 tn
Asignado True
Asignado a Camion 1
------
Camión 1
Carga Total 0 tn
Pedidos []
Costo Total 5000
Costo por tn None
Pedido J
Carga 6 tn
Asignado False
Asignado a Camion None
------
Camión 1
Carga Total 6 tn
Pedidos ['J']
Costo Total 8400
Costo por tn 1400.0
Pedido J
Carga 6 tn
Asignado True
Asignado a Camion 1


In [90]:
ruteo.get_camion(1).check_intercambio_pedido(ruteo.get_pedido("H"))

['A']

In [246]:
pedido = Pedido(ix="C", x=2, y=2, carga=11)

[pedido.distancia(other) > 2 for other in camion1.pedidos_asignados.values()]

[False, False]

In [247]:
any([pedido.distancia(other) > 2 for other in camion1.pedidos_asignados.values()])

False

In [250]:
not False and False

False

In [221]:
camion1.pedidos_asignados

{}

In [53]:
x = {}
x["A"] = Pedido("A")
x["B"] = Pedido("B")
print(x)
x.pop("B")
print(x)
x.clear()
print(x)

{'A': Pedido A, 'B': Pedido B}
{'A': Pedido A}
{}


In [238]:
camion1 = Camion(ix=1, carga_max=12, pedidos_max=3, dist_max=2)
camion2 = Camion(ix=2, carga_max=12, pedidos_max=3, dist_max=2)

camion1.add_pedido_checked(Pedido(ix="A", x=1, y=1, carga=4))
camion1.add_pedido_checked(Pedido(ix="B", x=3, y=3, carga=5))
camion2.add_pedido_checked(Pedido(ix="C", x=1, y=1, carga=11))

print(camion1.count_pedidos())
print(camion2.count_pedidos())
print(camion1.get_carga_total())
print(camion2.get_carga_total())
print(camion1.get_costo())
print(camion2.get_costo())
print(camion1.get_costo_tn())
print(camion2.get_costo_tn())


1
1
5
11
7000
11000
1400.0
1000.0


## Pruebas con datos reales

In [279]:
camion_prueba = df_camiones.iloc[0,:]
camion_prueba

camion           1
carga_max       12
clientes_max     3
dist_max         2
Name: 0, dtype: int64

In [280]:
camion = Camion(ix=camion_prueba.camion,
                carga_max=camion_prueba.carga_max,
                pedidos_max=camion_prueba.clientes_max,
                dist_max=camion_prueba.dist_max)

In [281]:
pedido_prueba = df_pedidos.iloc[0,0:2]
pedido_prueba

cliente_prueba = df_clientes.iloc[0]
cliente_prueba

Coord x    0.257571
Coord y    1.803726
Name: A, dtype: float64

Para controlar cómo copiar y guardar elementos en los camiones. Pensar qué pasaría si cargo muchos objetos en memoria.

In [284]:
# Pruebas de memoria para realizar cambios -> Ojo con modificar objetos.
camion1 = Camion("1", 2, 2, 2)
#camion2 = copy.copy(camion1)
camion2 = copy.deepcopy(camion1)
pedido1 = Pedido("A", 1, 1, 4)

print(camion1.pedidos_asignados.get("A").carga)
print(camion2.pedidos_asignados.get("A").carga)
print(pedido1.carga)

#pedido.carga = 5
#camion.pedidos_asignados.get("A").carga = 5
camion2.pedidos_asignados.get("A").carga = 5

print(camion1.pedidos_asignados.get("A").carga)
print(camion2.pedidos_asignados.get("A").carga)
print(pedido1.carga)

AttributeError: 'NoneType' object has no attribute 'carga'

In [175]:
class A(object):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
class B(A):
    
    def __init__(self):
        self.x = A.x

In [176]:
a = A(1,2)
b = B(a)

TypeError: __init__() takes 1 positional argument but 2 were given